In [12]:
# Import all the needed libraries
import pyterrier as pt
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from sklearn.ensemble import RandomForestRegressor
if not pt.started():
    pt.init()

In [13]:
# Load the dataset
dataset = pt.datasets.get_dataset("trec-deep-learning-passages")

#dataset = pt.get_dataset("trec-deep-learning-passages")
def msmarco_generate():
    print("Here")
    file = 'C:/Users/vrnsr/Downloads/collection/collection.tsv'
    with open(file, encoding="utf-8") as corpusfile:
        print("Here22")
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

#if index does not exist, then uncomment the following two lines            
#iter_indexer = pt.IterDictIndexer(r"C:\Users\vrnsr\Downloads\passage_index", meta=['docno', 'text'], meta_lengths=[20, 4096])
#index_ref = iter_indexer.index(msmarco_generate())

index_ref = pt.IndexRef.of(r"C:\Users\vrnsr\Downloads\passage_index\data.properties")

index = pt.IndexFactory.of(index_ref)

pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2", "WMODEL:Fasttext"])

train_topics = dataset.get_topics("train")
train_qrels = dataset.get_qrels("train")

11:46:16.630 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
11:46:16.639 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


In [14]:
train_topics = train_topics.sort_values(by='qid', ascending=True)[3:]
train_qrels = train_qrels.sort_values(by='qid', ascending=True)[3:]

In [15]:
# A simple join operation on the qids, as we have much more queries than qrels, we want to only use those
# queries with a qrel for optimal training performances.
temp = pd.merge(train_topics, train_qrels, left_on='qid', right_on='qid')

# Make the new train_topics dataframe with only those queries that have a qrel
train_topics_100 = pd.concat([temp['qid'], temp['query']], axis=1, keys=['qid', 'query'])

In [16]:
train_qrel_100 = pd.concat([temp['qid'], temp['docno'], temp['label']], axis=1, keys=['qid', 'docno', 'label'])

In [17]:
final_df = pd.read_csv('fastext_cosinesim.csv')
final_df_test = pd.read_csv('TestFastTextSim.csv')
final_df_test = final_df_test.rename(columns={'cosine_fastext':'cdist_fasttext'})
final_df_test['pid'] = final_df_test['pid'].apply(lambda x: int(x))

In [18]:
final_df

,qid,pid,cdist,cdist_fasttext
0,1000005,7871705,0.888864,0.709072
1,1000005,8564649,0.906684,-0.001293
2,1000005,8564641,0.936021,0.004588
3,1000005,1170305,0.866449,0.649741
4,1000005,635918,0.853606,0.440521
...,...,...,...,...
90995,1000162,252483,0.829435,0.691292
90996,1000162,1005526,0.837253,0.434586
90997,1000162,2229864,0.846573,0.417622
90998,1000162,2313262,0.752206,0.725946


In [19]:
final_df_test

,qid,pid,cdist,cdist_fasttext
0,156493,8182161,0.770682,0.568789
1,156493,6139386,0.753413,0.689835
2,156493,3288600,0.785007,0.635662
3,156493,3288596,0.762550,0.582584
4,156493,2259183,0.740538,0.593578
...,...,...,...,...
41995,146187,4239826,0.809611,0.828444
41996,146187,4874233,0.784742,0.786530
41997,146187,6065118,0.879775,0.834997
41998,146187,7392175,0.709208,0.828124


In [20]:
complete = pd.concat([final_df, final_df_test])
complete

,qid,pid,cdist,cdist_fasttext
0,1000005,7871705,0.888864,0.709072
1,1000005,8564649,0.906684,-0.001293
2,1000005,8564641,0.936021,0.004588
3,1000005,1170305,0.866449,0.649741
4,1000005,635918,0.853606,0.440521
...,...,...,...,...
41995,146187,4239826,0.809611,0.828444
41996,146187,4874233,0.784742,0.786530
41997,146187,6065118,0.879775,0.834997
41998,146187,7392175,0.709208,0.828124


In [21]:
def features(row):
    val = np.array(complete.loc[(complete['qid'] == int(row['qid'])) & (complete['pid'] == int(row['docid']))]['cdist_fasttext'].tolist())
    if len(val) == 0:
        return np.array([0])
    return val

In [22]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tf = pt.BatchRetrieve(index, wmodel="Tf")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")
pipeline = bm25 >> (tf ** pl2 ** (pt.apply.doc_features(features)))

In [23]:
#pipeline = (bm25 % 1000) >> (pt.apply.doc_features(features) ** tf ** pl2)

In [24]:
'''start = time.time()

rf = RandomForestRegressor(n_estimators=300)
rf_pipe = pipeline >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics_100.head(100), train_qrel_100.head(100))

end = time.time()

print(end - start)

import pickle
filename = 'rf_pipe_fasttext_model.sav'
pickle.dump(rf_pipe, open(filename, 'wb'))'''
#Takes 11402.762156486511 to train

11402.762156486511


In [25]:
'''pt.Experiment([bm25, rf_pipe], dataset.get_topics("test-2019"),
  dataset.get_qrels("test-2019"),eval_metrics=["ndcg", "map", "recip_rank"], save_dir=".", names=["BM25","LTR-FT-300"])'''

14:56:39.908 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


,name,ndcg,map,recip_rank
0,BM25,0.593433,0.370004,0.795028
1,LTR-FT-300,0.395238,0.099979,0.249274


In [44]:
import pickle
filename = 'rf_pipe_fasttext_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

pt.Experiment([bm25, loaded_model], dataset.get_topics("test-2019"),
  dataset.get_qrels("test-2019"),eval_metrics=["ndcg", "map", "recip_rank"], save_dir=".", names=["BM25","LTR-FT-300-saved"])

15:19:11.114 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
15:19:14.435 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
15:19:17.742 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
15:19:17.782 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


,name,ndcg,map,recip_rank
0,BM25,0.593433,0.370004,0.795028
1,LTR-FT-300-saved,0.439403,0.139369,0.451686


The following section is to perform L2R with summation of scores as a feature, and the only feature

In [63]:
#we now want to create a summation pipeline where we combine the scores

import numpy as np

#This method is used to give a new score to a document.
#In particular, it this notebook it is used to sum the bm25 score with the distilBERT score,
#which is multiplied by a certain factor to make it more compatible with the range of bm25 scores
def scores(row):
    docid = row["docid"]
    qid = row["qid"]
    num1 = 0

    val = np.array(complete.loc[(complete['qid'] == int(row['qid'])) & (complete['pid'] == int(row['docid']))]['cdist_fasttext'].tolist())
    if len(val) > 0:
        num1 = val

    new_score = row['score'] + float(num1)*10
    print(new_score)

    return (new_score)


bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tf = pt.BatchRetrieve(index, wmodel="Tf")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")
pipeline = bm25 >> (pt.apply.doc_features(features) ** tf ** pl2)
L2R = bm25 >> (tf ** pl2)
summation = bm25 >> pt.apply.doc_score(scores)

#use only feature from fastext and not pl2, 
#Transformer does not like qid to be int, so coverting it to string
df = complete.copy()

print(df.dtypes)
#df['docno'] = df['pid']
#df['score'] = df['cdist_fasttext']
df = df.rename(columns={"pid" : "docno", "cdist_fasttext":"score"})
#print(list(df))
df = pt.model.add_ranks(df.astype({'qid' : 'int', 'docno' : 'int'}))

print(df.dtypes)
print(list(df))

#df = pt.model.add_ranks(df.as)

df['qid'] = df['qid'].astype(str)
print(df.head())
fasttext_alone = pt.Transformer.from_df(df)


qid                 int64
pid                 int64
cdist             float64
cdist_fasttext    float64
dtype: object
qid        int32
docno      int32
cdist    float64
score    float64
rank       int32
dtype: object
['qid', 'docno', 'cdist', 'score', 'rank']
       qid    docno     cdist     score  rank
0  1000005  7871705  0.888864  0.709072    27
1  1000005  8564649  0.906684 -0.001293   973
2  1000005  8564641  0.936021  0.004588   936
3  1000005  1170305  0.866449  0.649741    98
4  1000005   635918  0.853606  0.440521   559


In [27]:
df.loc[df['qid'] == '1133167']

,qid,docno,cdist,score,rank
7028,1133167,5674622,0.766674,0.166368,964
7029,1133167,8160230,0.742514,0.733844,162
7030,1133167,8255705,0.776477,0.479741,571
7031,1133167,190809,0.756853,0.576262,406
7032,1133167,6467522,0.744870,0.744259,143
...,...,...,...,...,...
8027,1133167,3069212,0.755520,0.548734,452
8028,1133167,3423544,0.707982,0.802124,72
8029,1133167,3821786,0.813709,0.371274,756
8030,1133167,4273192,0.667993,0.399686,715


In [31]:
'''import time

start = time.time()

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=300)
pipe_fasttext = pipeline >> pt.ltr.apply_learned_model(rf)
pipe_fasttext.fit(train_topics_100[:100], train_qrel_100[:100])

end = time.time()

print(end - start)'''

'import time\n\nstart = time.time()\n\nfrom sklearn.ensemble import RandomForestRegressor\nrf = RandomForestRegressor(n_estimators=300)\npipe_fasttext = pipeline >> pt.ltr.apply_learned_model(rf)\npipe_fasttext.fit(train_topics_100[:100], train_qrel_100[:100])\n\nend = time.time()\n\nprint(end - start)'

In [54]:
df_results = pt.Experiment([bm25, summation, fasttext_alone, loaded_model], 
              dataset.get_topics("test-2019"), dataset.get_qrels("test-2019"), 
              filter_by_qrels = True, 
              perquery= True,
              eval_metrics=["map", "ndcg", "recip_rank"], 
              save_dir = ".", names = ["bm25", "fasttext_sum", "fast_alone","LTR-300-saved"])

16:12:02.011 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


In [55]:
df_results.to_csv("PerQueryFastTextAllModels.csv",index=False)

In [34]:
test_query = dataset.get_topics("test-2019")
print(test_query[test_query["qid"]=='443396'])

15:09:33.054 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8
        qid                query
142  443396  lps laws definition


In [45]:
query1 = "definition declaratory judgment"
query2 = "what are the three percenters"
query3 = "why did the us volunterilay enter ww1"
query_best = "definition declaratory judgment"

In [69]:
bm25.search(query_best, 130510)

,qid,docid,docno,rank,score,query
0,130510,1494936,1494936,0,41.166240,definition declaratory judgment
1,130510,7125239,7125239,1,40.377558,definition declaratory judgment
2,130510,799647,799647,2,39.952422,definition declaratory judgment
3,130510,7501563,7501563,3,39.800835,definition declaratory judgment
4,130510,996732,996732,4,39.395265,definition declaratory judgment
...,...,...,...,...,...,...
995,130510,7738711,7738711,995,16.203673,definition declaratory judgment
996,130510,691377,691377,996,16.199872,definition declaratory judgment
997,130510,837069,837069,997,16.199872,definition declaratory judgment
998,130510,3913776,3913776,998,16.199872,definition declaratory judgment


In [57]:
df_sum_q3 = summation.search(query3)
df_sum_q3.to_csv("FT_Summation_Query3.csv",index=False)

30.340022354623155
29.795791411290537
29.564491237072907
27.6762347362384
27.03193036403902
26.58871233690003
25.437240557396848
25.364802274901958
25.235233253486793
23.983590310188962
23.538403516787206
23.37200766626369
23.087605778349335
23.01904082614084
22.904697798579797
22.789905082652513
22.757231367969233
22.664257483367393
22.631170697834282
22.483009897743578
22.34505161940287
22.027522091608
21.917232514687598
21.70940798941628
21.70940798941628
21.70940798941628
21.52851597913199
21.510809717723888
21.305362734786357
21.305362734786357
21.131114396646844
21.10892803223922
21.10892803223922
21.10892803223922
21.10892803223922
21.10892803223922
20.845374412238705
20.748118415871904
20.748118415871904
20.726728600288208
20.54077241302697
20.37875868765089
20.351254095998218
20.351254095998218
20.257800350638515
20.178693762948274
20.178693762948274
20.139761686214392
20.08012718207173
20.022319688685574
20.022319688685574
20.0023994724105
20.0023994724105
19.905543519094223


11.188904538882019
11.188904538882019
11.181635282465109
11.181635282465109
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.170749090806066
11.15899994287531
11.152652466417184
11.152652466417184
11.152652466417184
11.152652466417184
11.152652466417184
11.152652466417184
11.152652466417184
11.14801070409998
11.13667618894141
11.132662081873196
11.132662081873196
11.132662081873196
11.132662081873196
11.132662081873196
11.1097182303615
11.1097182303615
11.1097182303615
11.1097182303615
11.1097182303615
11.1097182303615
11.1097182303615
11.1097182303615
11.1097182303615
11.1097182303615
11.10741470293303
11.10741470293303
11.106263297397676
11.106263297397676
11.106263297397676
11.105572568650931
11.0953241815544
11.0953241815544
11.0953241815544
11.0953241815544
11.0953241815544
11.0953241815544
11.08411599

In [46]:
df_sum_qbest = summation.search(query_best)
df_sum_qbest.to_csv("FT_Summation_BestQuery.csv",index=False)

41.16623959273336
40.37755754581451
39.95242166497842
39.8008345774324
39.395265152873236
38.66134694139317
38.66134694139317
37.66368789897934
37.222253459259846
35.88645011700757
34.47817709974436
34.37935045916029
34.37935045916029
34.111288294845906
33.79835518385679
33.76586461818342
32.766007612955924
32.69334990735618
32.36527447530863
31.65831134150973
31.221968235114204
31.16498530146699
30.85927197366889
29.773136261893498
29.773136261893498
29.31998500172886
28.912307627577768
28.853985496601027
28.50029410908722
28.50029410908722
28.50029410908722
28.275879096041372
27.80405385455048
27.710519136054277
27.522119729353204
27.498323073183762
27.331821490029867
26.963335015660185
26.255386957624186
26.156216191799857
25.915173172671324
25.67385880898153
25.583663490370057
25.43870356230104
25.43870356230104
25.26052809826512
25.26052809826512
24.813847862997562
24.37370636362195
24.179331535575503
23.95862860770795
23.95708500676042
23.922788889906162
23.922788889906162
23.841

17.07283322666153
17.060079038690397
17.060079038690397
17.05326409913459
17.05326409913459
17.046061184792332
17.046061184792332
17.026989646081038
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.00840707610098
17.001627212845598
16.997062240758854
16.997062240758854
16.997062240758854
16.997062240758854
16.997062240758854
16.997062240758854
16.997062240758854
16.983069610557838
16.97308897163561
16.965611187440548
16.92763908208722
16.9276390820

In [47]:
df_sum_q2 = summation.search(query2)
df_sum_q2.to_csv("FT_Summation_Query2.csv",index=False)

12.180420909027559
12.171076434781975
12.115600311099197
12.082614609000192
12.053412402339038
12.046962301325445
12.00932391497938
11.99542942096028
11.974647858954587
11.95984791710378
11.95426111040873
11.942884900190313
11.940171475607363
11.938815225631114
11.93745928372499
11.924731269064184
11.922507540014536
11.91369515635889
11.912732976961596
11.903196401840358
11.878459525548429
11.845636516197763
11.845636516197763
11.845636516197763
11.845636516197763
11.838179315461007
11.838179315461007
11.83259257358752
11.83259257358752
11.801577777018514
11.79998787328599
11.79998787328599
11.798928175433142
11.798928175433142
11.79817136491791
11.79817136491791
11.79760382074186
11.79760382074186
11.79760382074186
11.79680935062186
11.796520478925144
11.796520478925144
11.773073863270824
11.768970726052137
11.768970726052137
11.762821379998393
11.762821379998393
11.762821379998393
11.744411838262181
11.735473059277393
11.732170790481417
11.732170790481417
11.72824343193414
11.7282434

11.216149339019415
11.216149339019415
11.216149339019415
11.216149339019415
11.216149339019415
11.216149339019415
11.216149339019415
11.216149339019415
11.216149339019415
11.216149339019415
11.216149339019415
11.213277530067426
11.207996326870761
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.204984204080787
11.201401814370895
11.201401814370895
11.201401814370895
11.192934549421414
11.192934549421414
11.192934549421414
11.192934549421414
11.192934549421414
11.188121940917211
11.188121940917211
11.188121940917211
11.188121940917211
11.186424961153925
11.186424961153925
11.186424961153925
11.186424961153925
11.181664755440316
11.181664755440316
11.181664755440316
11.181664755440316
11.181664755440316
11.181664755440316
11.181664755440316
11.181664755440316
11.181664755440316
11.181664755440316
11.181664755

In [50]:
df_sum_q1 = summation.search(query1)
df_sum_q1.to_csv("FT_Summation_Query1.csv",index=False)

41.16623959273336
40.37755754581451
39.95242166497842
39.8008345774324
39.395265152873236
38.66134694139317
38.66134694139317
37.66368789897934
37.222253459259846
35.88645011700757
34.47817709974436
34.37935045916029
34.37935045916029
34.111288294845906
33.79835518385679
33.76586461818342
32.766007612955924
32.69334990735618
32.36527447530863
31.65831134150973
31.221968235114204
31.16498530146699
30.85927197366889
29.773136261893498
29.773136261893498
29.31998500172886
28.912307627577768
28.853985496601027
28.50029410908722
28.50029410908722
28.50029410908722
28.275879096041372
27.80405385455048
27.710519136054277
27.522119729353204
27.498323073183762
27.331821490029867
26.963335015660185
26.255386957624186
26.156216191799857
25.915173172671324
25.67385880898153
25.583663490370057
25.43870356230104
25.43870356230104
25.26052809826512
25.26052809826512
24.813847862997562
24.37370636362195
24.179331535575503
23.95862860770795
23.95708500676042
23.922788889906162
23.922788889906162
23.841

17.097705200289365
17.097705200289365
17.097705200289365
17.097705200289365
17.085748941391305
17.085748941391305
17.07283322666153
17.07283322666153
17.060079038690397
17.060079038690397
17.05326409913459
17.05326409913459
17.046061184792332
17.046061184792332
17.026989646081038
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01809447239944
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.01238431633336
17.00840707610098
17.001627212845598
16.997062240758854
16.997062240758854
16.997062240758854
16.997062240758854
16.997062240

In [52]:
df_ft_q1 = loaded_model.search(query1)
df_ft_q2 = loaded_model.search(query2)
df_ft_q3 = loaded_model.search(query3)
df_ft_qbest = loaded_model.search(query_best)
df_ft_q1.to_csv("FT_Query1.csv",index=False)
df_ft_q2.to_csv("FT_Query2.csv",index=False)
df_ft_q3.to_csv("FT_Query3.csv",index=False)
df_ft_qbest.to_csv("FT_QueryBest.csv",index=False)

df_ft_q1 = 0
df_ft_q2 = 0
df_ft_q3 = 0
df_ft_qbest = 0

In [75]:
df_ft_q1 = fasttext_alone.search(query1)
df_ft_q2 = fasttext_alone.search(query2)
df_ft_q3 = fasttext_alone.search(query3, qid = '1063750')
df_ft_qbest = fasttext_alone.search(query_best, qid = '130510')
df_ft_q1.to_csv("FTalone_Query1.csv",index=False)
df_ft_q2.to_csv("FTalone_Query2.csv",index=False)
df_ft_q3.to_csv("FTalone_Query3.csv",index=False)
df_ft_qbest.to_csv("FTalone_QueryBest.csv",index=False)

df_ft_q1 = 0
df_ft_q2 = 0
df_ft_q3 = 0
df_ft_qbest = 0

In [74]:
fasttext_alone.search(query_best, qid = '130510')

,qid,query,docno,cdist,score,rank
7,130510,definition declaratory judgment,1494935,0.617957,0.805637,0
5,130510,definition declaratory judgment,996740,0.614805,0.763225,1
199,130510,definition declaratory judgment,8247790,0.574245,0.739718,2
39,130510,definition declaratory judgment,4089773,0.598251,0.724759,3
297,130510,definition declaratory judgment,4261961,0.554546,0.718146,4
...,...,...,...,...,...,...
928,130510,definition declaratory judgment,4098008,0.678393,0.086261,995
909,130510,definition declaratory judgment,1812073,0.692682,0.085622,996
153,130510,definition declaratory judgment,6294918,0.574631,0.060323,997
108,130510,definition declaratory judgment,464597,0.666869,0.045527,998
